# Experiments with Naive RAG to find optimal k retrieved contexts from knowledge base

In [ ]:

import spacy
nlp = spacy.load("models/en_ner_bc5cdr_md-0.5.4")
print(nlp.pipe_names)


In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path(".").resolve()))
from utils.RAG_metadata import extract_keywords_and_entities, extract_age_gender
from utils.RAG_pinecone import init_index, RAGIndexer, pc
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv() 

/Users/lorenaraichle/Documents/clin_IQ/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:

index = init_index()
indexer = RAGIndexer(index)
indexer.describe_index()


ℹIndex 'rag-trainset-index' already exists.
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 411857}},
 'total_vector_count': 411857,
 'vector_type': 'dense'}


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 411857}},
 'total_vector_count': 411857,
 'vector_type': 'dense'}

In [1]:
from utils.RAG_preprocessing import DataPaths, DataStats
data = DataPaths.load("test_processed")

stats = DataStats.get_stats("test_processed", show_sources=False)

/Users/lorenaraichle/Documents/clin_IQ/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Stats for 'test_processed':
 - true_false: 42254 examples
 - multiple_choice: 43466 examples
 - short_answer: 13516 examples
 - multi_hop: 3941 examples
Total entries: 103177


In [4]:
from utils.RAG_naive_pipeline import RAGPipeline
from utils.prompt_utils import generate_model_input
import pprint
from datasets import Dataset, DatasetDict

import json
from utils.RAG_preprocessing import DataPaths

with open(DataPaths.paths["test_processed"], "r") as f:
    test_data_raw = json.load(f)

testset = DatasetDict({
    split: Dataset.from_list(split_data)
    for split, split_data in test_data_raw.items()
})
mc_testset = testset["multiple_choice"].to_list()


pprint.pprint(mc_testset[0])



{'correct_answer': 'B',
 'id': 'mc_0',
 'options': {'A': 'Caput',
             'B': 'Chignon',
             'C': 'Phlegmon',
             'D': 'None of the above.',
             'E': None},
 'question': 'Formation of one of the following is essential for success of '
             'this instrument?',
 'source': 'MC3-openlifescienceai/medmcqa',
 'type': 'multiple_choice'}


In [8]:
import json
import copy
from tqdm import tqdm
from utils.RAG_naive_pipeline import RAGPipeline
from utils.prompt_utils import generate_model_input
from utils.RAG_preprocessing import DataPaths

k_values = [3, 5, 8, 12]

for k in k_values:
    print(f"\n=== Running for k={k} ===")

    
    entries = test_data_raw["multiple_choice"]
    all_data_for_index = []
    for i, example in enumerate(entries):
        example["type"] = "multiple_choice"
        example["id"] = f"mc_{i}"
        all_data_for_index.append(example)
  
    pipeline = RAGPipeline(all_data_for_index, k=k)
    
    eval_examples = all_data_for_index[:4] # subset of questions
    
    predictions = []
    for example in tqdm(eval_examples, desc=f"Generating answers for k={k}"):
        prepared_example = generate_model_input(copy.deepcopy(example))
        response = pipeline.run(prepared_example)
        predictions.append(response)


    output_file_path = DataPaths.get_k_experiment_output_path(k)

    with open(output_file_path, 'w') as output_file:
        json.dump(predictions, output_file, indent=4)

    print(f"Saved results for k={k} to {output_file_path}")



=== Running for k=3 ===


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 33.96 GB, other allocations: 672.00 KB, max allowed: 36.27 GB). Tried to allocate 6.06 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).